# Predicción del precio de alquiler de una casa vacacional en Madrid

## Setup

In [ ]:
def getInputPath():
  return 'Inputs/'

In [ ]:
def getOutputPath():
  return 'Outputs/'

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.callbacks import EarlyStopping, ReduceLROnPlateau

import numpy
# fix random seed for reproducibility
numpy.random.seed(7)
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
%matplotlib inline

Using TensorFlow backend.


## Preparación de los datos

In [ ]:
datos_airbnb_consolidados = pd.read_csv(getOutputPath() + "consolidados_ml.csv")
# Paso 1: Preparamos los datos
features = datos_airbnb_consolidados.columns.drop(['price'])
X = datos_airbnb_consolidados[features].values

y = datos_airbnb_consolidados['price'].values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state=1)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size = 0.2, random_state=1)


print('Datos train: ', X_train.shape)
print('Datos validation: ', X_val.shape)
print('Datos test:  ', X_test.shape)
print('Propocion train:%0.3f'%np.mean(y_train))
print('Propocion validation:%0.3f'%np.mean(y_val))
print('Propocion test: %0.3f'%np.mean(y_test))

Datos train:  (10104, 211)
Datos validation:  (2527, 211)
Datos test:   (5414, 211)
Propocion train:74.229
Propocion validation:73.206
Propocion test: 74.547


In [ ]:

# Paso 1.1: Preprocesar las variables
scaler = StandardScaler().fit(X_train)
Xs_train = scaler.transform(X_train)
Xs_val = scaler.transform(X_val)
Xs_test = scaler.transform(X_test)

## Aproximación base

In [ ]:
# Compile model
model.compile(loss='mean_squared_error', optimizer='adam', metrics=['mse'])

In [ ]:
# create model
model = Sequential()
model.add(Dense(500, input_dim=211, kernel_initializer='random_normal'))
#model.add(Dropout(0.1))
model.add(Dense(500))
#model.add(Dropout(0.1))
model.add(Dense(500))

#model.add(Dropout(0.1))
model.add(Dense(500))

#model.add(Dropout(0.1))
model.add(Dense(500))

model.add(Dense(1))

In [ ]:
# Fit the model
early_stopper = EarlyStopping(patience=5)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.3,
                              patience=2, min_lr=0.0001)
model.fit(Xs_train, y_train, epochs=500, batch_size=500, validation_data=(Xs_val, y_val), callbacks=[ reduce_lr, early_stopper])

Train on 10104 samples, validate on 2527 samples
Epoch 1/500
10104/10104 [==============================] - 0s 24us/step - loss: 23.6767 - mean_squared_error: 2143.3873 - val_loss: 23.4037 - val_mean_squared_error: 2215.9860
Epoch 2/500
10104/10104 [==============================] - 0s 24us/step - loss: 23.6865 - mean_squared_error: 2148.3372 - val_loss: 23.7451 - val_mean_squared_error: 2194.9516
Epoch 3/500
10104/10104 [==============================] - 0s 24us/step - loss: 23.7252 - mean_squared_error: 2154.3525 - val_loss: 23.4579 - val_mean_squared_error: 2124.8104
Epoch 4/500
10104/10104 [==============================] - 0s 23us/step - loss: 23.0676 - mean_squared_error: 2096.0912 - val_loss: 22.5763 - val_mean_squared_error: 2088.2213
Epoch 5/500
10104/10104 [==============================] - 0s 23us/step - loss: 22.6758 - mean_squared_error: 2069.7340 - val_loss: 22.5737 - val_mean_squared_error: 2181.2577
Epoch 6/500
10104/10104 [==============================] - 0s 23us/step

In [ ]:
# evaluate the model
scores = model.evaluate(Xs_test, y_test)
print("\n%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))
scores

5414/5414 [==============================] - 0s 44us/step

mean_squared_error: 180298.89%


[1802.9888679721412, 1802.9888679721412]

## Búqueda de hiperparámetros con un algoritmo genético

In [ ]:
import sys
sys.path.insert(0, 'neural-network-genetic-algorithm/')


import main as ge


In [ ]:
import logging
from optimizer import Optimizer
from tqdm import tqdm

# Setup logging.
logging.basicConfig(
    format='%(asctime)s - %(levelname)s - %(message)s',
    datefmt='%m/%d/%Y %I:%M:%S %p',
    level=logging.DEBUG,
    filename='log.txt')

In [ ]:
generations = 20  # Number of times to evole the population.
population = 20  # Number of networks in each generation.


nn_param_choices = {
    'nb_neurons': [64, 128, 256, 512, 768, 1024],
    'nb_layers': [1, 2, 3, 4, 6, 7],
    'activation': ['linear','relu', 'elu', 'tanh', 'sigmoid'],
    'optimizer': ['rmsprop', 'adam', 'adagrad',
                  'adadelta', 'adamax', 'nadam'],
'batch_size': [100, 200, 300, 500, 1000]
}

logging.info("***Evolving %d generations with population %d***" %
             (generations, population))

best_network = ge.generate(generations, population, nn_param_choices, 1, Xs_train, Xs_val, Xs_test, y_train, y_val, y_test)


100%|██████████| 20/20 [07:59<00:00, 41.70s/it]


In [ ]:

best_network.network


{'activation': 'relu',
 'batch_size': 100,
 'nb_layers': 7,
 'nb_neurons': 768,
 'optimizer': 'adamax'}

In [ ]:
!ls 


drive  log.txt	sample_data


In [ ]:
from google.colab import files


files.download('log.txt')

## Afinar red con callbacks

### Base

In [ ]:
nb_layers = best_network.network['nb_layers']
nb_neurons = best_network.network['nb_neurons']
activation = best_network.network['activation']
optimizer = best_network.network['optimizer']


model = Sequential()

# Add each layer.
for i in range(nb_layers):

    # Need input shape for first layer.
    if i == 0:
        model.add(Dense(nb_neurons, activation=activation, input_dim=211))
    else:
        model.add(Dense(nb_neurons, activation=activation))

    

# Output layer.
model.add(Dense(1))

model.compile(loss='mean_squared_error', optimizer=optimizer,
              metrics=['mse'])

In [ ]:
model.fit(Xs_train, y_train,
              batch_size=best_network.network['batch_size'],
              epochs=150,
              verbose=2,
              validation_data=(Xs_val, y_val))

score = model.evaluate(Xs_test, y_test, verbose=2)
score

Train on 10104 samples, validate on 2527 samples
Epoch 1/150
 - 28s - loss: 3491.1083 - mean_squared_error: 3491.1083 - val_loss: 2448.1652 - val_mean_squared_error: 2448.1652
Epoch 2/150
 - 2s - loss: 1776.7720 - mean_squared_error: 1776.7720 - val_loss: 1717.4067 - val_mean_squared_error: 1717.4067
Epoch 3/150
 - 2s - loss: 1489.2175 - mean_squared_error: 1489.2175 - val_loss: 1626.3553 - val_mean_squared_error: 1626.3553
Epoch 4/150
 - 2s - loss: 1268.0068 - mean_squared_error: 1268.0068 - val_loss: 1611.8498 - val_mean_squared_error: 1611.8498
Epoch 5/150
 - 2s - loss: 1078.7451 - mean_squared_error: 1078.7451 - val_loss: 1557.7091 - val_mean_squared_error: 1557.7091
Epoch 6/150
 - 2s - loss: 877.7691 - mean_squared_error: 877.7691 - val_loss: 1583.9665 - val_mean_squared_error: 1583.9665
Epoch 7/150
 - 2s - loss: 734.7812 - mean_squared_error: 734.7812 - val_loss: 1531.5840 - val_mean_squared_error: 1531.5840
Epoch 8/150
 - 2s - loss: 523.1343 - mean_squared_error: 523.1343 - val_

[1315.0827249696786, 1315.0827249696786]

### Ajuste Learning Rate

In [ ]:
from keras.optimizers import Adamax

nb_layers = best_network.network['nb_layers']
nb_neurons = best_network.network['nb_neurons']
activation = best_network.network['activation']
optimizer = best_network.network['optimizer']

model = Sequential()

# Add each layer.
for i in range(nb_layers):

    # Need input shape for first layer.
    if i == 0:
        model.add(Dense(nb_neurons, activation=activation, input_dim=211))
    else:
        model.add(Dense(nb_neurons, activation=activation))

    

# Output layer.
model.add(Dense(1))
model.compile(loss='mean_squared_error', optimizer=optimizer,
              metrics=['mse'])

In [ ]:
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.3,
                              patience=2, min_lr=0.00001, cooldown=3)

model.fit(Xs_train, y_train,
              batch_size=best_network.network['batch_size'],
              epochs=150,
              verbose=2,
              validation_data=(Xs_val, y_val), callbacks=[reduce_lr])

score = model.evaluate(Xs_test, y_test, verbose=2)
score

Train on 10104 samples, validate on 2527 samples
Epoch 1/150
 - 30s - loss: 2532.8965 - mean_squared_error: 2532.8965 - val_loss: 1840.2206 - val_mean_squared_error: 1840.2206
Epoch 2/150
 - 2s - loss: 1656.9576 - mean_squared_error: 1656.9576 - val_loss: 1736.8250 - val_mean_squared_error: 1736.8250
Epoch 3/150
 - 2s - loss: 1388.4757 - mean_squared_error: 1388.4757 - val_loss: 1600.3117 - val_mean_squared_error: 1600.3117
Epoch 4/150
 - 2s - loss: 1182.2606 - mean_squared_error: 1182.2606 - val_loss: 1567.2190 - val_mean_squared_error: 1567.2190
Epoch 5/150
 - 2s - loss: 952.4171 - mean_squared_error: 952.4171 - val_loss: 1595.7205 - val_mean_squared_error: 1595.7205
Epoch 6/150
 - 2s - loss: 741.3650 - mean_squared_error: 741.3650 - val_loss: 1749.4547 - val_mean_squared_error: 1749.4547
Epoch 7/150
 - 2s - loss: 524.9158 - mean_squared_error: 524.9158 - val_loss: 1495.8945 - val_mean_squared_error: 1495.8945
Epoch 8/150
 - 2s - loss: 400.0002 - mean_squared_error: 400.0002 - val_lo

[1369.0175877772974, 1369.0175877772974]

## Conclusión
Como se ve la red neuronal con los siguientes parámetros es la que mejor resultado nos da:

{'activation': 'relu',

 'batch_size': 100,
 
 'nb_layers': 7,
 
 'nb_neurons': 768,
 
 'optimizer': 'adamax'}
 
 Nos da un valor de *MSE* de **1315.08**
 
